In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import numpy as np
import sys
import os
from tqdm import tqdm
import pickle

# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from src.ParObsSnakeEnv import ParObsSnakeEnv
from src.FullObsSnakeEnv import FullObsSnakeEnv
from src.utils import compute_metrics
import matplotlib.pyplot as plt

In [2]:
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super(DQN, self).__init__()
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(input_dim, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.fc3 = nn.Linear(self.hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# Define the DQN agent
class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, memory_size=10000, batch_size=64):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size

        self.memory = deque(maxlen=memory_size)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = DQN(state_dim, action_dim).to(self.device)
        self.target_model = DQN(state_dim, action_dim).to(self.device)
        self.target_model.load_state_dict(self.model.state_dict())
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.criterion = nn.MSELoss()

        # Metrics for plotting
        self.episode_rewards = []
        self.episode_losses = []
        self.epsilon_values = []
        self.average_q_values = []

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1), None
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        state = state.reshape((1, -1))
        with torch.no_grad():
            # print(">>>>", state.shape)
            q_values = self.model(state)
        return torch.argmax(q_values).item(), None

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)

        states = states.reshape((self.batch_size, -1))
        next_states = next_states.reshape((self.batch_size, -1))

        # Compute current Q-values
        q_values = self.model(states)
        q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

        # Compute target Q-values
        next_q_values = self.target_model(next_states).max(1)[0]
        target_q_values = rewards + (1 - dones) * self.gamma * next_q_values

        # Update the Q-network
        loss = self.criterion(q_values, target_q_values)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Log the loss
        self.episode_losses.append(loss.item())

        # Track average Q-value
        self.average_q_values.append(q_values.mean().item())

        # Decay epsilon
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def save(self, filename):
        """Saves the entire agent to a file."""
        state = {
            'model_state_dict': self.model.state_dict(),
            'target_model_state_dict': self.target_model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'hyperparameters': {
                'state_dim': self.state_dim,
                'action_dim': self.action_dim,
                'gamma': self.gamma,
                'epsilon': self.epsilon,
                'epsilon_decay': self.epsilon_decay,
                'epsilon_min': self.epsilon_min,
                'batch_size': self.batch_size,
            },
            'memory': list(self.memory),  # Convert deque to list
        }
        with open(filename, 'wb') as f:
            pickle.dump(state, f)
        print(f"Agent saved to {filename}")
    
    @classmethod
    def load(cls, filename, lr=0.001):
        """Loads the agent from a file."""
        with open(filename, 'rb') as f:
            state = pickle.load(f)
        
        # Recreate the agent
        agent = cls(
            state['hyperparameters']['state_dim'],
            state['hyperparameters']['action_dim'],
            lr=lr,
            gamma=state['hyperparameters']['gamma'],
            epsilon=state['hyperparameters']['epsilon'],
            epsilon_decay=state['hyperparameters']['epsilon_decay'],
            epsilon_min=state['hyperparameters']['epsilon_min'],
            batch_size=state['hyperparameters']['batch_size'],
        )
        # Restore the agent's state
        agent.model.load_state_dict(state['model_state_dict'])
        agent.target_model.load_state_dict(state['target_model_state_dict'])
        agent.optimizer.load_state_dict(state['optimizer_state_dict'])
        agent.memory = deque(state['memory'], maxlen=len(state['memory']))
        print(f"Agent loaded from {filename}")
        return agent

    # Train the agent
    def train(self, env, episodes=1000, update_target_every=10, save_plots=False, plots_path='dqn_training_plots.png'):
        for episode in tqdm(range(episodes), desc="Training", unit='episode'):
            state = env.reset()
            total_reward = 0
            done = False

            while not done:
                action, _ = self.choose_action(state)
                next_state, reward, done, _ = env.step(action)
                self.remember(state, action, reward, next_state, done)
                self.replay()
                state = next_state
                total_reward += reward

            self.episode_rewards.append(total_reward)
            self.epsilon_values.append(self.epsilon)

            if (episode + 1) % update_target_every == 0:
                self.update_target_model()

        if save_plots:
            self.save_plots(plots_path)
    
    def save_plots(self, plots_path):
        plots_dir = os.path.dirname(plots_path)
        os.makedirs(plots_dir, exist_ok=True)

        fig, axs = plt.subplots(2, 2, figsize=(15, 10))

        # Rewards per episode
        axs[0, 0].plot(self.episode_rewards)
        axs[0, 0].set_title("Episode Rewards")
        axs[0, 0].set_xlabel("Episode")
        axs[0, 0].set_ylabel("Total Reward")

        # Loss per episode
        axs[0, 1].plot(self.episode_losses)
        axs[0, 1].set_title("Loss Over Training")
        axs[0, 1].set_xlabel("Episode")
        axs[0, 1].set_ylabel("Loss")

        # Epsilon decay
        axs[1, 0].plot(self.epsilon_values)
        axs[1, 0].set_title("Epsilon Decay")
        axs[1, 0].set_xlabel("Episode")
        axs[1, 0].set_ylabel("Epsilon Value")

        # Average Q-values
        axs[1, 1].plot(self.average_q_values)
        axs[1, 1].set_title("Average Q-Values")
        axs[1, 1].set_xlabel("Episode")
        axs[1, 1].set_ylabel("Average Q-Value")

        plt.tight_layout()
        plt.savefig(plots_path)
        plt.close()

In [4]:
grid_size = 10
# env = FullObsSnakeEnv(grid_size=grid_size, interact=False)
env = ParObsSnakeEnv(grid_size=grid_size, interact=False)
if isinstance(env, FullObsSnakeEnv):
    state_dim = env.observation_space.shape[0] * env.observation_space.shape[1] * env.observation_space.shape[2]
else:
    state_dim = env.observation_space.shape[0]

action_dim = env.action_space.n
agent = DQNAgent(state_dim, action_dim)

In [5]:
num_episodes = 300
agent.train(env, episodes=300)

Training:   0%|          | 0/300 [00:00<?, ?episode/s]/tmp/ipykernel_24247/1154075020.py:46: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  states = torch.FloatTensor(states).to(self.device)
Training: 100%|██████████| 300/300 [01:09<00:00,  4.33episode/s]


In [6]:
environment = 'full 'if isinstance(env, FullObsSnakeEnv) else 'par'

agent_name = f'dqn_agent_{environment}_{num_episodes}_{grid_size}.pkl'
model_weights_dir = os.path.join('../..', 'models', 'dqn')
os.makedirs(model_weights_dir, exist_ok=True)
agent_path = os.path.join(model_weights_dir, agent_name)

agent.save(agent_path)
# agent = DQNAgent.load(agent_path)

Agent saved to ../../models/dqn/dqn_agent_par_300_10.pkl


In [7]:
if isinstance(env, ParObsSnakeEnv):
    env = ParObsSnakeEnv(grid_size=2*grid_size, interact=False)

model_metrics_dir = os.path.join('../..', 'artifacts', 'models_stats', 'dqn')
os.makedirs(model_metrics_dir, exist_ok=True)

train_metrics_name = f'dqn_train_metrics_{environment}_{num_episodes}_{grid_size}.png'
train_metrics_path = os.path.join(model_metrics_dir, train_metrics_name)
agent.save_plots(train_metrics_path)

num_simulations = 100
sim_metrics_name = f'dqn_sim_metrics_{environment}_{num_episodes}_{env.grid_size}_{num_simulations}.jsn'
sim_metrics_path = os.path.join(model_metrics_dir, sim_metrics_name)
compute_metrics(agent, env, sim_metrics_path, num_simulations=num_simulations)

  4%|▍         | 4/100 [00:00<00:03, 27.11it/s]

Snake length: 14, Episode reward: 1085
Snake length: 12, Episode reward: 883
Snake length: 13, Episode reward: 957
Snake length: 27, Episode reward: 2175
Snake length: 26, Episode reward: 2108


  7%|▋         | 7/100 [00:00<00:04, 20.69it/s]

Snake length: 26, Episode reward: 2157
Snake length: 10, Episode reward: 742
Snake length: 14, Episode reward: 1083
Snake length: 45, Episode reward: 3785


 12%|█▏        | 12/100 [00:00<00:04, 17.71it/s]

...

 96%|█████████▌| 96/100 [00:07<00:00, 13.44it/s]

Snake length: 34, Episode reward: 2945
Snake length: 21, Episode reward: 1699
Snake length: 21, Episode reward: 1701


100%|██████████| 100/100 [00:07<00:00, 13.23it/s]

Snake length: 41, Episode reward: 3380
Snake length: 30, Episode reward: 2456
Snake length: 15, Episode reward: 1151


{'snake_lengths': [14,
  12,
  13,
  27,
  26,
  26,
  10,
  14,
  45,
  19,
  23,
  21,
  24,
  19,
  35,
  18,
  15,
  53,
  23,
  26,
  15,
  34,
  18,
  25,
  25,
  24,
  63,
  2,
  15,
  9,
  24,
  29,
  17,
  28,
  35,
  27,
  30,
  23,
  59,
  35,
  35,
  51,
  42,
  15,
  16,
  26,
  25,
  28,
  53,
  18,
  35,
  29,
  23,
  19,
  35,
  38,
  27,
  24,
  38,
  48,
  28,
  18,
  25,
  27,
  35,
  34,
  22,
  39,
  39,
  25,
  12,
  15,
  16,
  32,
  28,
  15,
  22,
  33,
  33,
  60,
  21,
  13,
  40,
  20,
  17,
  11,
  25,
  45,
  48,
  15,
  15,
  22,
  32,
  24,
  34,
  21,
  21,
  41,
  30,
  15],
 'episode_rewards': [1085,
  883,
  957,
  2175,
  2108,
  2157,
  742,
  1083,
  3785,
  1471,
  1891,
  1664,
  1947,
  1483,
  2902,
  1514,
  1186,
  4559,
  1892,
  2128,
  1139,
  2837,
  1411,
  2012,
  2078,
  1896,
  5485,
  26,
  1124,
  655,
  1896,
  2398,
  1390,
  2305,
  2980,
  2227,
  2473,
  1857,
  5074,
  2911,
  2959,
  4337,
  3509,
  1193,
  1220,
  2234,
  2

In [8]:
if isinstance(env, FullObsSnakeEnv):
    env.interact = True
else:
    env = ParObsSnakeEnv(grid_size=2*grid_size)
    
state = env.reset()
done = False
while not done:
    action, _ = agent.choose_action(state)
    state, reward, done, _ = env.step(action)
    env.render()
    print(f"Reward: {reward}")

Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 76
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Reward: 1
Rewar